# Signature Name Mapping Function

The purpose of this notebook is to derive a function that can replace the global dictionary, <i>signature_name_mapping</b>.

## Allen's Interval Algebra

The folowing code cells were copied from the "derive_allens_algebra" notebook, so that I can have access to the 13 FourPointNetwork objects from which Allen's algebra was derived.

In [1]:
import os
import qualreas as qr
import numpy as np

import sys

In [2]:
sys.setrecursionlimit(10000)

In [3]:
path = os.path.join(os.getenv('PYPROJ'), 'qualreas')

In [4]:
pt_alg = qr.Algebra(os.path.join(path, "Algebras/Linear_Point_Algebra.json"))

In [5]:
less_than_rel = '<'

In [6]:
base_nets = qr.generate_consistent_networks(pt_alg, lessthan=less_than_rel, verbose=False)


13 consistent networks


## Function Definition

So, now I have the 13 FourPointNetworks corresponding to the relations in Allen's interval algebra.

In [7]:
base_nets.keys()

dict_keys(['B', 'M', 'O', 'FI', 'DI', 'S', 'E', 'SI', 'D', 'F', 'OI', 'MI', 'BI'])

In [10]:
def get_2x2_subarray(arr):
    """Return the 4 elements of the upper right 2x2 subarray."""
    return arr[0][2], arr[0][3], arr[1][2], arr[1][3]

In [11]:
for relname in base_nets.keys():
    ptnet = base_nets[relname]
    ptarr = np.array(ptnet.to_list())
    print(f"{relname}: {get_2x2_subarray(ptarr)}")

B: ('<', '<', '<', '<')
M: ('<', '<', '=', '<')
O: ('<', '<', '>', '<')
FI: ('<', '<', '>', '=')
DI: ('<', '<', '>', '>')
S: ('=', '<', '>', '<')
E: ('=', '<', '>', '=')
SI: ('=', '<', '>', '>')
D: ('>', '<', '>', '<')
F: ('>', '<', '>', '=')
OI: ('>', '<', '>', '>')
MI: ('>', '=', '>', '>')
BI: ('>', '>', '>', '>')


In [25]:
# Intervals: X = (x1, x2), Y = (y1, y2)
#
#              0     1       2      3
#         |   x1    x2      y1     y2
#---------|-------------------------------
#  0  x1  |    =     <     x1y1  x1y2
#  1  x2  |    >     =     x2y1  x2y2
#  2  y1  |  y1x1  y1x2      =     <
#  3  y2  |  y2x1  y2x2      >     =

def signature_name_mapping_fnc(ptnet):
    ptarr = np.array(ptnet.to_list())
    
    x2y1 = ptarr[1][2]
    # x2 < y1, then Before
    if x2y1 == "<":
        return "B"
    # x2 = y1, then Meets
    elif x2y1 == "=":
        return "M"
    else:
        pass
    
    y2x1 = ptarr[3][0]
    # y2 < x1, then After
    if y2x1 == "<":
        return "BI"
    # y2 = x1, then Met-By
    elif y2x1 == "=":
        return "MI"
    else:
        pass
    
    # y1 < x2 < y2
    if (ptarr[2][1] == "<") and (ptarr[1][3] == "<"):
        x1y1 = ptarr[0][2]
        # x1 < y1, then Overlaps
        if x1y1 == "<":
            return "O"
        # x1 = y1, then Starts
        elif x1y1 == "=":
            return "S"
        # x1 > y1, then During
        else:
            return "D"
            
    # x1 < y2 < x2
    if (ptarr[0][3] == "<") and (ptarr[3][1] == "<"):
        y1x1 = ptarr[2][0]
        # y1 < x1, then Overlapped-By
        if y1x1 == "<":
            return "OI"
        # y1 = x1, then Started-By
        elif y1x1 == "=":
            return "SI"
        # y1 > x1, then Contains
        else:
            return "DI"
    
    # x2 = y2
    if ptarr[1][3] == "=":
        # x1 = y1, then Equals
        if ptarr[0][2] == "=":
            return "E"
        # y1 < x1 < y2, then Finishes
        elif (ptarr[2][0] == "<") and (ptarr[0][3] == "<"):
            return "F"
        else:
            pass
        
    # y2 = x2
    if ptarr[3][1] == "=":
        # y1 = x1, then Equals
        if ptarr[2][0] == "=":
            return "E"
        # x1 < y1 < x2, then Finished-By
        elif (ptarr[0][2] == "<") and (ptarr[2][1] == "<"):
            return "FI"
        else:
            pass
        
    print("ERROR: If we got to this statement, then something is wrong here.")
    
    return None

## Function Test

In [32]:
print("Current Name : Derived Name")
print("---------------------------")
for relname in base_nets.keys():
    ptnet = base_nets[relname]
    ptarr = np.array(ptnet.to_list())
    print(f"{relname} : {signature_name_mapping_fnc(ptnet)}")

Current Name : Derived Name
---------------------------
B : B
M : M
O : O
FI : FI
DI : DI
S : S
E : E
SI : SI
D : D
F : F
OI : OI
MI : MI
BI : BI
